# 導入函式庫

In [1]:
import requests
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
#資料欄位統一型別
def align_format(df,empty_column=True):
  df["總床數"] = df["總床數"].astype(int)
  df["佔床數"] = df["佔床數"].astype(int)
  if empty_column:
    df["空床數"] = df["總床數"] - df["佔床數"]
  else:
    df["空床數"] = df["空床數"].astype(int)
  df["佔床率"] = round(df["佔床數"]/df["總床數"],4)
  df['佔床率'] = df['佔床率'].apply(lambda x: '{:.2%}'.format(x))
  return df

### 奇美醫院各區

In [14]:
def ChimeiScraper(number):
    url = "https://www.chimei.org.tw/%E4%BD%94%E5%BA%8A%E7%8E%87%E6%9F%A5%E8%A9%A2/%E4%BD%94%E5%BA%8A%E7%8E%87%E6%9F%A5%E8%A9%A2.aspx?ihospital=10&ffloor="
    form_data = {'RBL院區': str(number), 'Btn查詢': '查詢'}
    response = requests.post(url, data=form_data)
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        table = soup.find("table", {"id": "DG1"})
        data = []
        rows = table.find_all("tr")
        for row in rows[1:]:
            cells = row.find_all("td")
            row_data = [cell.get_text(strip=True) for cell in cells]
            data.append(row_data)
        df = pd.DataFrame(data, columns=["病床類別", "總床數", "佔床數", "空床數", "佔床率"])
        df = align_format(df,False)
        return df
    else:
        print(f'請求失敗 (奇美醫院, 區域 {number}):', response.status_code)
        return None

In [15]:
#奇美醫院:10 柳營奇美醫院:13 佳里奇美醫院:14
df_Chime_Chime = ChimeiScraper(10)
df_Chime_Liuying = ChimeiScraper(13)
df_Chime_Jiali = ChimeiScraper(14)

In [16]:
df_Chime_Chime

,病床類別,總床數,佔床數,空床數,佔床率
0,日間照護床,50,49,1,98.00%
1,精神科病床,40,38,2,95.00%
2,急性一般病床（單人房）,108,107,1,99.07%
3,急性一般病床（雙人房）,324,322,2,99.38%
4,急性一般病床（健保房）,435,431,4,99.08%
5,急性加護病床,109,92,17,84.40%
6,嬰兒床,20,9,11,45.00%
7,嬰兒病床,28,17,11,60.71%
8,安寧病床(健保房),14,13,1,92.86%
9,安寧病床(單人房),1,1,0,100.00%


In [17]:
df_Chime_Chime.dtypes

病床類別    object
總床數      int32
佔床數      int32
空床數      int32
佔床率     object
dtype: object

### 郭綜合醫院

In [18]:
def KuoScraper():
    url = "https://www.kgh.com.tw/InstantMessage/BedInfoAPI"
    response = requests.get(url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        table = soup.find("table")
        data = []
        rows = table.find_all("tr")
        for row in rows[1:]:
            cells = row.find_all("td")
            row_data = [cell.get_text(strip=True) for cell in cells]
            data.append(row_data)
        df = pd.DataFrame(data, columns=["病床類別", "總床數", "佔床數", "空床數", "佔床率"])
        df = df.iloc[1:, :].reset_index(drop=True)
        df = align_format(df,True)
        return df
    else:
        print('請求失敗 (郭綜合醫院):', response.status_code)

In [19]:
df_Kuo = KuoScraper()
df_Kuo

,病床類別,總床數,佔床數,空床數,佔床率
0,急性健保病床,124,76,48,61.29%
1,急性一般差額病床,112,41,71,36.61%
2,急性加護病床,22,11,11,50.00%
3,新生兒急性加護病房,2,1,1,50.00%
4,急性新生兒中重度病床,8,3,5,37.50%
5,亞急性呼吸照護病房,12,0,12,0.00%
6,慢性呼吸照護病房,38,34,4,89.47%
7,負壓隔離病房,2,1,1,50.00%


In [20]:
df_Kuo.dtypes

病床類別    object
總床數      int32
佔床數      int32
空床數      int32
佔床率     object
dtype: object

### 安南醫院

In [21]:
def AnnanScraper():
    url = "https://www.tmanh.org.tw/Announce/CurrentBedAvailability"
    response = requests.get(url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        table = soup.find("table")
        data = []
        rows = table.find_all("tr")
        for row in rows[1:]:
            cells = row.find_all("td")
            row_data = [cell.get_text(strip=True) for cell in cells]
            data.append(row_data)
        df = pd.DataFrame(data, columns=["病床類別", "總床數", "佔床數", "空床數"])
        df = df.iloc[1:, :].reset_index(drop=True)
        df = df.iloc[:9,:]
        df = align_format(df,False)
        return df
    else:
        print('請求失敗 (安南醫院):', response.status_code)

In [22]:
df_Annan = AnnanScraper()
df_Annan

,病床類別,總床數,佔床數,空床數,佔床率
0,急性一般精神病床,40,20,20,50.00%
1,急性加護病床,62,41,21,66.13%
2,急性收差額病床,125,85,40,68.00%
3,新生兒中重度病床,3,0,3,0.00%
4,嬰兒床,16,2,14,12.50%
5,急性保護隔離病床,4,2,2,50.00%
6,安寧病房(不收差額),7,0,7,0.00%
7,安寧病房(收差額),3,0,3,0.00%
8,慢性一般精神病床,228,215,13,94.30%


In [23]:
df_Annan.dtypes

病床類別    object
總床數      int32
佔床數      int32
空床數      int32
佔床率     object
dtype: object

### 台南醫院

In [24]:
def TainanScraper():
    url = "https://www.tmh.org.tw/tmh2016/ImpBD.aspx?Kind=2"
    response = requests.get(url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        table = soup.find("table", {"id": "ctl00_ContentPlaceHolder1_GV_Bed"})
        data = []
        rows = table.find_all("tr")
        for row in rows[1:]:
            cells = row.find_all("td")
            row_data = [cell.get_text(strip=True) for cell in cells]
            data.append(row_data)
        df = pd.DataFrame(data, columns=["病床類別", "總床數", "佔床數", "空床數", "佔床率"])
        df = align_format(df,False)
        return df
    else:
        print('請求失敗 (台南醫院):', response.status_code)

In [25]:
df_Tainan = TainanScraper()
df_Tainan

,病床類別,總床數,佔床數,空床數,佔床率
0,急性一般病床(非收差額病床),179,139,40,77.65%
1,急性一般病床(收差額病床),167,120,47,71.86%
2,普通隔離病床,6,1,5,16.67%
3,加護病床,40,23,17,57.50%
4,亞急性呼吸照護病床,12,3,9,25.00%
5,精神急性一般病床,24,17,7,70.83%
6,嬰兒病床,3,0,3,0.00%


In [26]:
df_Tainan.dtypes

病床類別    object
總床數      int32
佔床數      int32
空床數      int32
佔床率     object
dtype: object

### 成大醫院

In [28]:
def NCKUHScraper():
  url = "https://web.hosp.ncku.edu.tw/nckm/Bedstatus/BedStatus.aspx"
  response = requests.get(url)
  if response.status_code == 200:
    soup = BeautifulSoup(response.text, 'html.parser')
    table = soup.find("table", {"id": "GV_EmgInsure"})
    data = []
    rows = table.find_all("tr")
    for row in rows[1:]:
      cells = row.find_all("td")
      row_data = [cell.get_text(strip=True) for cell in cells]
      data.append(row_data)
    df = pd.DataFrame(data, columns=["病床類別", "總床數", "佔床數", "空床數"])
    df = align_format(df,True)
    return df
  else:
    print('請求失敗 (成大醫院):', response.status_code)

In [29]:
df_NCKUH = NCKUHScraper()
df_NCKUH

,病床類別,總床數,佔床數,空床數,佔床率
0,急性一般病床,645,635,10,98.45%
1,急性精神病床,30,26,4,86.67%
2,成人加護病床,98,90,8,91.84%
3,小兒加護病床,8,3,5,37.50%
4,新生兒加護病床,20,18,2,90.00%
5,燒傷加護病床,10,8,2,80.00%
6,亞急性呼吸照護病床,12,3,9,25.00%
7,嬰兒病床,23,20,3,86.96%
8,安寧病床(不收差額),12,12,0,100.00%
9,其他特殊病床,61,60,1,98.36%


In [30]:
df_NCKUH.dtypes

病床類別    object
總床數      int32
佔床數      int32
空床數      int32
佔床率     object
dtype: object

### 新樓醫院

In [11]:
#driver操作
def driver_operate(driver,url):
    driver.get(url)
    driver.implicitly_wait(10)
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    table = soup.find('table')
    df = pd.read_html(str(table))[0]
    return df

#資料清理
def clean_df(df,rows_to_drop):
    df = df.drop(rows_to_drop)
    df.iloc[0] = df.iloc[0].shift(-1).fillna(method='ffill')
    df = df.iloc[:, :-1]
    df.reset_index(drop=True, inplace=True)
    df = df[1:]
    df = df.drop(df.columns[-1], axis=1)
    df.columns = ["病床類別", "總床數", "佔床數", "空床數", "佔床率"]
    return df
    

def SinlauScraper(area):
    chrome_options = Options()
    user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.77 Safari/537.36"
    chrome_options.add_argument(f'user-agent={user_agent}')
    driver = webdriver.Chrome(options=chrome_options)
    driver.maximize_window()
    try:
        if area == "Tainan":
            url = "http://59.125.233.216/sinlau/pgm/imbed/IMBED1.asp"
            df = driver_operate(driver, url)
            rows_to_drop = [0, 1, 3, 7, 9, 17, 18]
        elif area == "Madou":
            url = "http://59.125.233.216/sinlau/pgm/imbed/IMBED1.asp?SLArea=SL201"
            df = driver_operate(driver, url)
            rows_to_drop = [0, 1, 3, 7, 12, 13]
        else:
            return None
    finally:
        driver.quit()
    df = clean_df(df,rows_to_drop)
    df = align_format(df,False)
    return df

In [12]:
df_csinlau_Tainan = SinlauScraper("Tainan")
df_csinlau_Madou = SinlauScraper("Madou")

In [27]:
df_csinlau_Tainan

,病床類別,總床數,佔床數,空床數,佔床率
1,急性自付差額病床(單人房),62,33,29,53.23%
2,急性自付差額病床(雙人房),58,27,31,46.55%
3,急性一般病床(健保床),165,106,59,64.24%
4,慢性日間照護病床(健保床),20,14,6,70.00%
5,安寧病床(單人房),3,1,2,33.33%
6,安寧病床(雙人房),12,10,2,83.33%
7,嬰兒床,20,3,17,15.00%
8,負壓隔離病床,2,1,1,50.00%
9,加護病床,32,17,15,53.12%
10,嬰兒病床,10,7,3,70.00%


In [13]:
df_csinlau_Tainan.dtypes

病床類別    object
總床數      int32
佔床數      int32
空床數      int32
佔床率     object
dtype: object